Import Pyserial Package (docs: https://pyserial.readthedocs.io/en/latest/)

In [1]:
import serial
from pathlib import Path
import time

# Configuration
Function which allows user to configure:
- Serial Comm Port Selection
- Number of Sensors Connected
- Device Label (lab name etc.) - will determine the path of the output file.

In [2]:
def config():
    
    invalid_chars = ['<','>', ':', '/', "\\",'?', "*"]
    
    # User inputs the Arduino Comm Port
    while True:
        try:
            n_port = int(input("\nSelect Comm Port: "))
            port = f'COM{abs(n_port)}'
            print(f"'{port}' selected.")
            break
        except ValueError:
            print("\nError... Comm Port Input must be an integer. Please Re-Input Below:")
    
    # User inputs the number of sensors
    while True:
        try:
            n_sensors = int(input("\nNumber of Enviroment Sensors: "))
            break
        except ValueError:
            print("\nError... Input must be an integer. Please Re-Input Below:")
    
    # User inputs label for device
    while True:
        tag = input("\nMonitor Name: ")
        if any(char in tag for char in invalid_chars):
            print(f"Error... Must not contain illegal characters:")
            print(" ".join(invalid_chars))
            print("Please Re-Input Below:")
        else:
            break
            
    while True: 
        query = input("\nConfirm? (Y/N): ")
        if query == "Y" or query == "y":
            break
        elif query == "N" or query == "n":
            [port, n_sensors, tag] = config()
            return [port, n_sensors, tag]
            
    # Could add another input of where to save data???
    
    # Writes user-inputs to a .txt and returns a list of the user-inputs
    config_file = open('config/config.txt', 'w')  # write new CSV if file doesn't exist
    config_file.write(f"Config Params: Serial Port, Number of Enviorment Sensors, Device Label\n{port}\n{n_sensors}\n{tag}")
    config_file.close()
    print(f"\nConfig File Written to '{path_config}' succesfully.")
    return [port, n_sensors, tag]

# Monitor Function
Function that uses the pyserial lib to communicate with the Arduino.
Checks if device is reset and updates value labels for use in the sketch.

Reads each line of the serial monitor which corresponds to a timetamp, label, value and units (for each sensor) respectively

For example, a device which has 3 sensors and a timestamp associated to an onboard RTC will output 10 lines. (3 * n_sensors + 1):

Note: Sensor names (_relhumid, _temp, _press) are already defined in the arduino sketch.

In [3]:
def write_headers(ser, path, n_sensors):
    data_list = []
    for i in range(3*3 + 1 + 1):  # 3 sensors connected (hard coded for now), 3 columns, first line is time and
        # read arduino serial comms, decode from ASCII to python default 'utf'and strip formatting characters
        ser_line = ser.readline().decode('utf').strip("\r\n")
        data_list.append(ser_line)  # append the comms to a list
        
    if not Path(path).exists():
        print(f"\nWriting New File to '{path}'")
        file = open(path, 'w')  # write new CSV if file doesn't exist
    else:
        print(f"\nWriting New Headers to '{path}'")
        file = open(path, 'r+')  # else append the first line
        
    print("\nAdalogger Time :", data_list[0])

    headers = ['Time Stamp']
    for i in range(n_sensors):
        headers.append(data_list[3*i+1] + " ("  + data_list[3*i+3] + ")")
        print(data_list[i*3+1], ":", data_list[i*3+2], 
              data_list[i*3+3])

    headers = ','.join(headers)
    file.write("%s\n" % headers)  
    # first line written is the headers if new file created
    
    file.close()
    file = open(path, 'a+')
    csv_list = [data_list[0]]
    for i in range(n_sensors):
        csv_list.append(data_list[i*3+2])
    csv_line = ','.join(csv_list)    
    file.write(str(csv_line))  # write data to a CSV
    file.write("\n")
    ser.close()
    return file

In [4]:
def temp_humid_monitor(port, baud_rate, path, n_sensors, tag):
    try:
        # Define the serial object
        ser = serial.Serial(port, baud_rate)
        
        # Close and Reopen the Serial Port
        ser.close()
        ser.open()   
        print("\n\nRunning...\n")
        tag = f'{tag}\n'  # sketch reads the written data up till the newline character.
        
        # Check if file already exists
        # If not, write new file and read comms to find tag names and create headers.
        if not Path(path).exists():
            ser.write(bytes(tag, 'ASCII')) # write the label to the arduino to start printing data
            file = write_headers(ser, path, n_sensors)

        else:
            file = open(path, 'a+')  # otherwise append to existing CSV
        
        ser.close()
        ser.open()
        # Start Monitoring Loop
        while True:
            # in the case of updated tags while device running, need to re-write the tags each loop itt
            ser.write(bytes(tag, 'ASCII')) 
            data_list = []
            for i in range((3*3 + 1 + 1)):
                # read arduino serial, decode and strip any formatting characters
                ser_line = ser.readline().decode('utf').strip("\r\n")
                data_list.append(ser_line)
            print("\nAdalogger Time : ", data_list[0])  # first index is the time step
            csv_list = [data_list[0]]
            for i in range(n_sensors):
                print(data_list[i*3+1], ":", data_list[i*3+2], 
                      data_list[i*3+3])
                csv_list.append(data_list[i*3+2])
            csv_line = ','.join(csv_list)

            file.write(str(csv_line))  # write data to a CSV
            file.write("\n")
            file.close()  # closes file
            file = open(path, 'a+')
            ser.close()
            ser.open()
            time.sleep(0.5)
    
    # Close port if script interrupted
    except KeyboardInterrupt:
        file.close()
        ser.close()
        raise KeyboardInterrupt

# Main Script
The main program which executes the respective functions in order.

In [ ]:
baud_rate = 9600  # pre-defined baud rate
path_config = 'config/config.txt'  # path to save the config file

# User-input to determine wether to re-configure the device or re-load previous data.
while True:
    query = input("Configure Device? (Y/N): ")
    if query == "Y" or query == "y":
        [port, n_sensors, tag] = config()
        break
        
    elif query == "N" or query == "n":
        if not Path(path_config).exists():
            print("Previous config. not found. Running setup:")
            [port, n_sensors, tag] = config()
            n_sensors = int(n_sensors)
        else:
            config_file = open(path_config, 'r')
            config_data = config_file.readlines()
            config_file.close()
            [port, n_sensors, tag] = [config_data[i].strip("\n") for i in range(1,len(config_data))]
            n_sensors = int(n_sensors)
            print(f'\nPrevious Config Found...\nPort: {port}\nNumber of Enviroment Sensors: {n_sensors}\nLabel: {tag}')
            while True:
                query = input("\nIs this correct? (Y/N): ")
                if query == "Y" or query == "y":
                    break
                elif query == "N" or query == "n":
                    [port, n_sensors, tag] = config()
                    break
        break

path = f"data/{tag}_data.csv"  # can allow user to define the path to save data. Hard coded for now.

while True:
    try: 
        temp_humid_monitor(port, baud_rate, path, n_sensors, tag)
    except KeyboardInterrupt:
        while True:
            query = input("\nInterrupted. Re-Configure? (Y/N): ")
            if query == "Y" or query == "y":
                n_sensors_old = n_sensors
                tag_old = tag
                [port, n_sensors, tag] = config()
                path = f"data/{tag}_data.csv"  # can allow user to define the path to save data. Hard coded for now.
                if n_sensors > n_sensors_old and tag == tag_old:
                    ser = serial.Serial(port, baud_rate) 
                    tag = f'{tag}\n'  # sketch reads the written data up till the newline character.
                    ser.write(bytes(tag, 'ASCII')) # write the label to the arduino to start printing data
                    ser = write_headers(ser, path, n_sensors)
                    ser.close()
                break
            elif query == "N" or query == "n":
                print(f'\nPrevious Config Found...\nPort: {port}\nNumber of Enviroment Sensors: {n_sensors}\nLabel: {tag}')
                while True:
                    query = input("\nIs this correct? (Y/N): ")
                    if query == "Y" or query == "y":
                        break
                    elif query == "N" or query == "n":
                        [port, n_sensors, tag] = config()
                        break
                break

Configure Device? (Y/N):  y

Select Comm Port:  4


'COM4' selected.



Number of Enviroment Sensors:  3

Monitor Name:  lab_3

Confirm? (Y/N):  y



Config File Written to 'config/config.txt' succesfully.


Running...


Writing New File to 'data/lab_3_data.csv'

Adalogger Time : 2021/6/28 12:3:41
lab_3_relhumid : 44.31 %
lab_3_temp : 24.05 degC
lab_3_press : 101138.36 Pa

Adalogger Time :  2021/6/28 12:3:43
lab_3_relhumid : 44.33 %
lab_3_temp : 24.04 degC
lab_3_press : 101137.61 Pa

Adalogger Time :  2021/6/28 12:3:45
lab_3_relhumid : 44.43 %
lab_3_temp : 24.05 degC
lab_3_press : 101137.66 Pa

Adalogger Time :  2021/6/28 12:3:47
lab_3_relhumid : 44.61 %
lab_3_temp : 24.05 degC
lab_3_press : 101138.25 Pa

Adalogger Time :  2021/6/28 12:3:49
lab_3_relhumid : 44.50 %
lab_3_temp : 24.05 degC
lab_3_press : 101138.08 Pa

Adalogger Time :  2021/6/28 12:3:51
lab_3_relhumid : 44.41 %
lab_3_temp : 24.05 degC
lab_3_press : 101138.30 Pa

Adalogger Time :  2021/6/28 12:3:53
lab_3_relhumid : 44.33 %
lab_3_temp : 24.06 degC
lab_3_press : 101138.42 Pa

Adalogger Time :  2021/6/28 12:3:55
lab_3_relhumid : 44.28 %
lab_3_temp : 24.04 degC
lab_3_pres


Interrupted. Re-Configure? (Y/N):  Y

Select Comm Port:  4


'COM4' selected.



Number of Enviroment Sensors:  3

Monitor Name:  lab_3

Confirm? (Y/N):  Y



Config File Written to 'config/config.txt' succesfully.


Running...


Adalogger Time :  2021/6/28 12:26:29
lab_3_relhumid : 46.20 %
lab_3_temp : 23.99 degC
lab_3_press : 101111.01 Pa

Adalogger Time :  2021/6/28 12:26:31
lab_3_relhumid : 46.18 %
lab_3_temp : 24.00 degC
lab_3_press : 101110.84 Pa

Adalogger Time :  2021/6/28 12:26:33
lab_3_relhumid : 46.18 %
lab_3_temp : 24.00 degC
lab_3_press : 101110.61 Pa

Adalogger Time :  2021/6/28 12:26:35
lab_3_relhumid : 46.18 %
lab_3_temp : 24.01 degC
lab_3_press : 101110.55 Pa

Adalogger Time :  2021/6/28 12:26:37
lab_3_relhumid : 46.20 %
lab_3_temp : 24.01 degC
lab_3_press : 101110.19 Pa

Adalogger Time :  2021/6/28 12:26:39
lab_3_relhumid : 46.20 %
lab_3_temp : 24.01 degC
lab_3_press : 101109.95 Pa

Adalogger Time :  2021/6/28 12:26:41
lab_3_relhumid : 46.22 %
lab_3_temp : 24.01 degC
lab_3_press : 101110.09 Pa

Adalogger Time :  2021/6/28 12:26:43
lab_3_relhumid : 46.22 %
lab_3_temp : 24.02 degC
lab_3_press : 101110.11 Pa

Adalogger Time :